## Stats507 Homework 3, Winter 2020
### Yifeng He
#### yifengh@umich.edu

__Remark: For this homework, do not use "input" in your code!!!__

I discussed this homework with Wang Xiang.

Problem 1 took me 3 hours,Problem 2 took me 4 hours,Problem 3 took me 3.5 hours
Problem 4 took me 1 hours,Problem 5 took me 2 hours,Problem 6 took me 1 hours 

In [1]:
# you might need to install pycodestyle_magic and flake8
%load_ext pycodestyle_magic
%pycodestyle_on

### Problem 1: Counting Word Bigrams
1. Write a function count_bigrams_in_file that takes a filename as its only argument.

In [35]:
import string


def count_bigrams_in_file(fname):
    if not isinstance(fname, str):
        raise TypeError("The file name should be string")
    count = {}
    word = ""
    words = ""
    try:
        fhand = open(fname)
    except FileNotFoundError:
        raise FileNotFoundError("The input file cannot be opened")
    else:
        for line in fhand:
            line = line.lower()
            for item in line:
                if item not in string.punctuation:
                    words += item
            words = words.split()
            if len(word) != 0 and len(words) != 0:
                if (word, words[0]) in count:
                    count[(word, words[0])] += 1
                else:
                    count[(word, words[0])] = 1
            for i in range(len(words)-1):
                if (words[i], words[i+1]) in count:
                    count[(words[i], words[i+1])] += 1
                else:
                    count[(words[i], words[i+1])] = 1
            if len(words) != 0:
                word = words[-1]
            words = ""
        fhand.close()
        return count

In [36]:
with open('./test.txt', 'w') as f:
    f.write('Half a league, half a league,\n Half a league onward.')

count_bigrams_in_file("test.txt") == {('a', 'league'): 3,
                                      ('half', 'a'): 3,
                                      ('league', 'half'): 2,
                                      ('league', 'onward'): 1}

True

2. Run your function on this file, and pickle the resulting dictionary in a file called mb.bigrams.pickle.<br>

In [37]:
import pickle
fname = "WandP.txt"
dic = count_bigrams_in_file(fname)
pickle.dump(dic, open("mb.bigrams.pickle", "wb"))

3. Write a function collocations that takes a file name as its only argument and returns a dictionary.<br>

In [38]:
def collocations(fname):
    if type(fname) != str:
        raise NameError("The file name should be string")
    count = {}
    words = ""
    try:
        fhand = open(fname)
    except FileNotFoundError:
        raise FileNotFoundError("The input file cannot be opened")
    else:
        content = fhand.read()
        fhand.close()
        content = content.lower()
        for item in content:
            if item not in string.punctuation:
                words += item
        words = words.split()
        for i in range(0, len(words)):
            if i == 0:
                if words[i] in count:
                    count[words[i]].add(words[i+1])
                else:
                    count[words[i]] = set()
                    count[words[i]].add(words[i+1])
            elif i == (len(words) - 1):
                if words[i] in count:
                    count[words[i]].add(words[i-1])
                else:
                    count[words[i]] = set()
                    count[words[i]].add(words[i-1])
            else:
                if words[i] in count:
                    count[words[i]].update([words[i-1], words[i+1]])
                else:
                    count[words[i]] = set()
                    count[words[i]].update([words[i-1], words[i+1]])
        return count

In [39]:
collocations('test.txt') == {'half': {'league', 'a'},
                             'a': {'half', 'league'},
                             'league': {'onward', 'half', 'a'},
                             'onward': {'league'}}

True

4. Run your function on the file WandP.txt and pickle the resulting dictionary in a file called mb.colloc.pickle.

In [40]:
fname = "WandP.txt"
col = collocations(fname)
pickle.dump(col, open("mb.colloc.pickle", "wb"))

### Problem 2: More Fun with Vectors
#### Define the class Vector in one code cell below, since class can't be declared several times.<br>

In [41]:
class Vector:
    def __init__(self, dims=0, entrs=()):
        if not isinstance(dims, int):
            raise TypeError("dimension should be integer type")
        if dims < 0:
            raise ValueError("dimension should be a positive number")
        if not isinstance(entrs, (list, tuple)):
            raise TypeError("entrs should be list or tuple")
        if len(entrs) != dims:
            raise ValueError("Entered dimension and"
                             " vector's dimension don't agree.")
        self.dim = dims
        self.entr = entrs

    def __gt__(self, other):
        if self.dim != other.dim:
            raise ValueError("vectors should have"
                             " same dimensions for comparison.")
        for i in range(self.dim):
            if self.entr[i] != other.entr[i]:
                return self.entr[i] > other.entr[i]
        return False

    def __lt__(self, other):
        if self.dim != other.dim:
            raise ValueError("vectors should have same"
                             " dimensions for comparison.")
        for i in range(self.dim):
            if self.entr[i] != other.entr[i]:
                return self.entr[i] < other.entr[i]
        return False

    def __eq__(self, other):
        if self.dim != other.dim:
            raise ValueError
            ("vectors should have same"
             " dimensions for comparison.")
        for i in range(self.dim):
            if self.entr[i] != other.entr[i]:
                return False
        return True

    def __ge__(self, other):
        return self.entr > other.entr or self.entr == other.entr

    def __le__(self, other):
        return self.entr < other.entr or self.entr == other.entr

    def dot(self, other):
        if not isinstance(other, Vector):
            raise TypeError("The input should be vector type.")
        if self.dim != other.dim:
            raise ValueError("The two vectors should agree in dimension.")
        sum = 0
        for i in range(self.dim):
            sum += self.entr[i]*other.entr[i]
        return sum

    def __mul__(self, other):
        if isinstance(other, (float, int)):
            tup = (other,)*self.dim
            other = Vector(self.dim, tup)
        if self.dim != other.dim:
            raise ValueError("Two vectors' dimension don't agree.")
        lst = []
        for i in range(self.dim):
            lst.append(self.entr[i]*other.entr[i])
        return Vector(self.dim, tuple(lst))

    def __rmul__(self, other):
        return self.__mul__(other)

    def norm(self, p):
        sum = 0
        max = float("-inf")
        if p < 0:
            raise ValueError("The norm number should be positive.")
        if p == 0:
            for i in range(self.dim):
                if self.entr[i] != 0:
                    sum += 1
            return sum
        elif p == float('inf'):
            for i in range(self.dim):
                if self.entr[i] > max:
                    max = self.entr[i]
            return max
        else:
            for i in range(self.dim):
                sum += (abs(self.entr[i])**p)**(1/p)
            return sum

**2.2**<br>

I choose tuple because list is mutable, which makes changing elements inside vectors easily. However, the correct way is to use class method to change elements inside vectors.  

**2.3**<br>

Dimension and entries are instance attributes.The reason we use instance attributes here is that we may need to create different objects under Vector class.

In [42]:
# 2.1
print(isinstance(Vector(3, (1, 2, 3)), Vector))

# 2.4
print((Vector(3, (1, 2, 3)) > Vector(3, (1, 2, 2))) == (True))
print((Vector(3, (1, 2, 3)) >= Vector(3, (1, 2, 2))) == (True))
print((Vector(3, (1, 2, 3)) < Vector(3, (1, 2, 2))) == (False))
print((Vector(3, (1, 2, 3)) <= Vector(3, (1, 2, 2))) == (False))
print((Vector(3, (1, 2, 3)) == Vector(3, (1, 2, 2))) == (False))

# 2.5
print((Vector(3, (1, 2, 3)).dot(Vector(3, (1, 2, 2)))) == 11.0)

# 2.6
print(2 * Vector(3, (1, 2, 3)) == Vector(3, (2, 4, 6)))
print(Vector(3, (1, 2, 3)) * 2 == Vector(3, (2, 4, 6)))

# 2.7
print(Vector(3, (1, 2, 3)).norm(1) == 6.0)

###  Problem 3: Iterators and Generators
1. Define a class Fibo of iterators that enumerate the Fibonacci numbers.

In [43]:
class Fibo():
    def __init__(self):
        self.n1 = 1
        self.n2 = 0

    def __next__(self):
        (self.n1, self.n2, oldn2) = (self.n1+self.n2, self.n1, self.n2)
        return (oldn2)

    def __iter__(self):
        return self

In [44]:
ans = Fibo()
[next(ans) for _ in range(10)] == [0, 1, 1, 2, 3, 5, 8, 13, 21, 34]

True

2. Define a class GenFibo of iterators that enumerate generalized Fibonacci numbers.<br>

In [ ]:
class GenFibo(Fibo):
    def __init__(self, n1=0, n2=1):
        self.n1 = n2
        self.n2 = n1

In [ ]:
ans = GenFibo(2, 1)
[next(ans) for _ in range(10)] == [2, 1, 3, 4, 7, 11, 18, 29, 47, 76]

3. Define a generator primes that enumerates the prime numbers.
#### You may use the function is_prime that we defined in class (or something similar to it), but such solutions will not receive full credit<br>

In [7]:
import math


def primes():
    n = 1
    while True:
        n += 1
        while any(n % x == 0 for x in range(2, int(math.sqrt(n))+1)):
            n += 1
        yield n

In [8]:
ans = primes()
[next(ans) for _ in range(10)] == [2, 3, 5, 7, 11, 13, 17, 19, 23, 29]

True

4. Define a generator ulam that enumerates the Ulam numbers.

In [3]:
def ulam():
    u1 = 1
    u2 = 2
    t = [u1, u2]
    n = 0
    while True:
        n += 1
        while not is_ulam(t, n):
            n += 1
        if n > 2:
            t.append(n)
        yield n


def is_ulam(t, u):
    count = 0
    t_set = set(t)
    for num in t:
        if (u-num) in t_set and (u-num) != num:
            count += 1
        if count == 3:
            return False
    return True

In [4]:
ans = ulam()
[next(ans) for _ in range(10)] == [1, 2, 3, 4, 6, 8, 11, 13, 16, 18]

True

###  Problem 4: List Comprehensions and Generator Expressions (5 points)
1. Write a list comprehension that enumerates the sequence $3^n - 1$ for n = 1, 2, 3, ..., 20.

In [5]:
pow3minus1 = [3**n - 1 for n in range(1, 21)]

2. Write a generator expression that enumerates the Lazy Caterer's sequence. Please assign this generator expression to a variable called caterer.<br>

In [34]:
def lazy_cut():
    h = -1
    while True:
        h += 1
        yield h


caterer = ((h**2+h+2)//2 for h in lazy_cut())

3. Write a generator expression that enumerates the pyramid numbers.<br>

In [6]:
def pyramid_num():
    h = 0
    while True:
        h += 1
        yield h


pyramid = (h*(h+1)*(2*h+1)//6 for h in pyramid_num())

4. Write a generator expression that enumerates the octahedral numbers.

In [5]:
def oct_num():
    h = 0
    while True:
        h += 1
        yield h


octa = (h*(2*(h**2)+1)//3 for h in oct_num())

### Problem 5: Map, Filter and Reduce
1. Write a one-line expression that computes the sum of the first 10 odd  (starting from 1).<br>

In [2]:
import itertools
import functools

In [5]:
sum_of_odd_numbers = functools.reduce(lambda x, y: x+y, list(filter(lambda x: x%2 != 0, range(20))))

1:80: E228 missing whitespace around modulo operator
1:80: E501 line too long (100 > 79 characters)


2. Write a one-line expression that computes the product of the first 13 primes. You may use the primes generator that you defined above. Please assign the output of this expression to a variable called product_of_primes.<br>

In [28]:
product_of_primes = functools.reduce(lambda x, y:x*y,list(itertools.islice(primes(), 0, 13)))

ValueError: too many values to unpack (expected 3)

3. Write a one-line expression that computes the sum of the squares of the first 31 primes. You may use the primes generator that you defined above. Please assign the output of this expression to a variable called squared_primes.

In [29]:
squared_primes = functools.reduce(lambda x, y:x+y,list(map(lambda x:x*x,list(itertools.islice(primes(),0,31)))))

ValueError: too many values to unpack (expected 3)

4. Write a one-line expression that computes a list of the first twenty harmonic numbers. Please assign the output of this expression to a variable called harmonics.

In [32]:
harmonics = list(itertools.accumulate(range(1, 21), lambda x, y:x+1/y))

ValueError: too many values to unpack (expected 3)

5. Write a one-line expression that computes the geometric mean of the first 12 octahedral numbers.

In [34]:
octa_geom = functools.reduce(lambda x, y:(x*y)**(1/12), [next(octa) for _ in range(12)])

ValueError: too many values to unpack (expected 3)

### Problem 6: Fun with Polynomials
1. Write a function make_poly that takes a list of numbers (ints and/or foats) coeffs as its only argument and returns a function p.

In [38]:
def make_poly(coeffs):
    def get_num(x):
        sum = 0
        for i in range(len(coeffs)):
            sum += coeffs[i]*(x**i)
        return sum
    return get_num

make_poly([1, 3, 4])(2)

23

In [39]:
make_poly([1, 3, 4])(2) == 23

True

2. Write a function eval_poly that takes two lists of numbers (ints and/or foats), coeffs and args.

In [40]:
def eval_poly(coeffs,args):
    return list(map(make_poly(coeffs),args))

eval_poly([1, 3, 4], [2, 3])

[23, 46]

In [41]:
eval_poly([1, 3, 4], [2, 3]) == [23, 46]

True